### m8.1stan

m8.1stan is the first model in the Statistical Rethinking book (pp. 249) using Stan.

Here we will use Turing's NUTS support, which is currently (2018) the originalNUTS by [Hoffman & Gelman]( http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf) and not the one that's in Stan 2.18.2, i.e., Appendix A.5 in: https://arxiv.org/abs/1701.02434

The StatisticalRethinking pkg uses, e.g., Turing, CSV, DataFrames

In [1]:
using StatisticalRethinking

loaded


┌ Warning: Package Turing does not have CmdStan in its dependencies:
│ - If you have Turing checked out for development and have
│   added CmdStan as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Turing
│ Loading CmdStan into Turing from project dependency, future warnings for Turing are suppressed.
└ @ nothing nothing:840


Read in rugged data as a DataFrame

In [2]:
d = CSV.read(joinpath(dirname(Base.pathof(StatisticalRethinking)), "..", "data",
    "rugged.csv"), delim=';');
# Show size of the DataFrame (should be 234x51)
size(d)

(234, 51)

Apply log() to each element in rgdppc_2000 column and add it as a new column

In [3]:
d = hcat(d, map(log, d[Symbol("rgdppc_2000")]));

Rename our col x1 => log_gdp

In [4]:
rename!(d, :x1 => :log_gdp);

Now we need to drop every row where rgdppc_2000 == missing

When this (https://github.com/JuliaData/DataFrames.jl/pull/1546) hits DataFrame it'll be conceptually easier: i.e., completecases!(d, :rgdppc_2000)

In [5]:
notisnan(e) = !ismissing(e)
dd = d[map(notisnan, d[:rgdppc_2000]), :];

Updated DataFrame dd size should equal 170 x 52

In [6]:
size(dd)

(170, 52)

Define the Turing model

In [7]:
@model m8_1stan(y, x₁, x₂) = begin
    σ ~ Truncated(Cauchy(0, 2), 0, Inf)
    βR ~ Normal(0, 10)
    βA ~ Normal(0, 10)
    βAR ~ Normal(0, 10)
    α ~ Normal(0, 100)

    for i ∈ 1:length(y)
        y[i] ~ Normal(α + βR * x₁[i] + βA * x₂[i] + βAR * x₁[i] * x₂[i], σ)
    end
end;

Test to see that the model is sane. Use 2000 for now, as in the book.
Need to set the same stepsize and adapt_delta as in Stan...

In [8]:
posterior = sample(m8_1stan(dd[:,:log_gdp], dd[:,:rugged], dd[:,:cont_africa]),
    Turing.NUTS(2000, 1000, 0.95));
# Describe the posterior samples
describe(posterior)

┌ Warning: Indexing with colon as row will create a copy in the future. Use `df[col_inds]` to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1
┌ Warning: Indexing with colon as row will create a copy in the future. Use `df[col_inds]` to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1
┌ Warning: Indexing with colon as row will create a copy in the future. Use `df[col_inds]` to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1
┌ Info: [Turing] looking for good initial eps...
└ @ Turing /Users/rob/.julia/packages/Turing/pRhjG/src/samplers/support/hmc_core.jl:246
[NUTS{Union{}}] found initial ϵ: 0.000390625
└ @ Turing /Users/rob/.julia/packages/Turing/pRhjG/src/samplers/support/hmc_core.jl:291
[NUTS] Sampling...  0%  ETA: 2:54:35
  ϵ:         0.000390625
  α:         1.0
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling...  1%  ETA: 0:15:01
  ϵ:         0.0295168

4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 39%  ETA: 0:01:15
  ϵ:         0.031159626916970055
  α:         0.9387613371440396
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 40%  ETA: 0:01:14
  ϵ:         0.03065621862743999
  α:         0.6828343824299761
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 40%  ETA: 0:01:14
  ϵ:         0.02771121501638729
  α:         0.9885648629271666
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 41%  ETA: 0:01:13
  ϵ:         0.029360699358661077
  α:         0.9918172439387035
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 42%  ETA: 0:01:12
  ϵ:         0.022465199369414032
  α:         0.9298696907441051
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 42%  ETA: 0:01:12
  ϵ:         0.019864477955201992
  α:         0.962511803312884
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 43%  ETA: 0:01:11
  ϵ:         0.030764926576999554
  α:         

[NUTS] Sampling... 82%  ETA: 0:00:21
  ϵ:         0.03023450596349138
  α:         0.9986517932356251
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 83%  ETA: 0:00:20
  ϵ:         0.03023450596349138
  α:         0.9824975703202119
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 84%  ETA: 0:00:19
  ϵ:         0.03023450596349138
  α:         0.994755549334208
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 85%  ETA: 0:00:17
  ϵ:         0.03023450596349138
  α:         0.8978355193403604
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 86%  ETA: 0:00:16
  ϵ:         0.03023450596349138
  α:         1.0
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 87%  ETA: 0:00:15
  ϵ:         0.03023450596349138
  α:         0.9782115668707878
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 88%  ETA: 0:00:14
  ϵ:         0.03023450596349138
  α:         0.7892128080551941
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[

[NUTS] Finished with
  Running time        = 112.03881808599999;
  #lf / sample        = 0.005;
  #evals / sample     = 43.475;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0, 1.0].


[NUTS] Sampling...100% Time: 0:01:53


Iterations = 1:2000
Thinning interval = 1
Chains = 1
Samples per chain = 2000

Empirical Posterior Estimates:
              Mean          SD         Naive SE        MCSE         ESS    
       α    9.15457105  0.765130661 0.01710884169 0.07924062096   93.234248
  lf_num    0.00500000  0.223606798 0.00500000000 0.00500000000 2000.000000
      βA   -1.92307929  0.382801534 0.00855970252 0.04065865973   88.642342
      βR   -0.18856012  0.194242868 0.00434340257 0.01959817447   98.233343
       σ    0.97896795  0.304171309 0.00680147723 0.02712915683  125.708285
 elapsed    0.05601941  0.092950553 0.00207843755 0.00312959032  882.122658
 epsilon    0.03045781  0.013589897 0.00030387932 0.00038033925 1276.703410
eval_num   43.47500000 25.318666877 0.56614260237 0.61718524900 1682.870099
     βAR    0.39865371  0.204083981 0.00456345655 0.00612705177 1109.467882
      lp -253.23689788 59.063186553 1.32069300100 4.68070434807  159.225026
  lf_eps    0.03045781  0.013589897 0.00030387932 0.00

Output reg. params of interest:

In [9]:
m_08.1t_turing_result = "
        Mean           SD        Naive SE        MCSE         ESS
α    9.2140454953  0.416410339 0.00931121825 0.0303436655  188.324543
βA  -1.9414588557  0.373885658 0.00836033746 0.0583949856   40.994586
βR  -0.1987645549  0.158902372 0.00355316505 0.0128657961  152.541295
σ    0.9722532977  0.440031013 0.00983939257 0.0203736871  466.473854
βAR  0.3951414223  0.187780491 0.00419889943 0.0276680621   46.062071
"

LoadError: syntax: extra token "0.1" after end of expression

Here's the map2stan output:

In [10]:
m_08.1_map2stan_result = "
       Mean StdDev lower 0.89 upper 0.89 n_eff Rhat
 a      9.24   0.14       9.03       9.47   291    1
 bR    -0.21   0.08      -0.32      -0.07   306    1
 bA    -1.97   0.23      -2.31      -1.58   351    1
 bAR    0.40   0.13       0.20       0.63   350    1
 sigma  0.95   0.05       0.86       1.03   566    1
""#-

LoadError: syntax: extra token "0.1" after end of expression

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*